NEURAL NETWORKS - CNN for Image Classification,Text Summarizer,ChatBOT

In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.7 MB/s eta 0:00:00


IMAGE CLASSIFICATION

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import gradio as gr

# Load the MobileNetV2 model (pretrained on ImageNet)
base_model = keras.applications.MobileNetV2(weights="imagenet")

# Preprocessing function for MobileNetV2
def preprocess_image(img):
    img = img.resize((224, 224))  # Resize to match MobileNetV2 input
    img = np.array(img) / 255.0   # Normalize
    img = img.reshape(1, 224, 224, 3)
    return img

# Prediction function
def predict_image(img):
    img = preprocess_image(img)
    prediction = base_model.predict(img)
    decoded_prediction = keras.applications.mobilenet_v2.decode_predictions(prediction, top=3)[0]
    return {item[1]: float(item[2]) for item in decoded_prediction}

# Create Gradio interface
interface = gr.Interface(fn=predict_image, inputs=gr.Image(type="pil"), outputs=gr.Label())

# Launch Gradio app
interface.launch(share=True)  # 'share=True' creates a public link

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2df3f3701f18061ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


IMAGE CLASSIFICATION IN RATING FORMAT



In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import gradio as gr

# Load MNIST dataset
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize images (scale pixel values between 0 and 1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build a simple ANN model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),  # Flatten 28x28 images into 1D
    keras.layers.Dense(128, activation='relu'),  # Hidden layer with 128 neurons
    keras.layers.Dense(10, activation='softmax') # Output layer for 10 classes (digits 0-9)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Function to preprocess input image
def preprocess_image(image):
    image = image.convert("L").resize((28, 28))  # Convert to grayscale and resize
    image = np.array(image) / 255.0  # Normalize
    image = image.reshape(1, 28, 28)  # Reshape for model input
    return image

# Prediction function for Gradio
def predict_digit(image):
    image = preprocess_image(image)
    prediction = model.predict(image)
    return {str(i): float(prediction[0][i]) for i in range(10)}

# Gradio Interface
interface = gr.Interface(fn=predict_digit, inputs=gr.Image(type="pil"), outputs=gr.Label())

# Launch Gradio App
interface.launch(share=True)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8794 - loss: 0.4320 - val_accuracy: 0.9597 - val_loss: 0.1386
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.9628 - loss: 0.1265 - val_accuracy: 0.9682 - val_loss: 0.1042
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9758 - loss: 0.0803 - val_accuracy: 0.9743 - val_loss: 0.0844
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9830 - loss: 0.0581 - val_accuracy: 0.9772 - val_loss: 0.0720
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9877 - loss: 0.0414 - val_accuracy: 0.9765 - val_loss: 0.0773
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c2bdcfb194cd604507.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co

TEXT SUMMARIZER



In [ ]:
!pip install google-generativeai


In [ ]:
!pip install --upgrade google-generativeai


In [ ]:
import gradio as gr
from transformers import pipeline

# Load a free pre-trained summarization model from Hugging Face
summarizer = pipeline("summarization", model="t5-small")  # You can also use "facebook/bart-large-cnn"

# Define the summarization function
def summarize_text(text):
    if len(text) < 20:
        return "Input text is too short to summarize!"

    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Create the Gradio interface
interface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(label="Enter text to summarize", lines=10),
    outputs=gr.Textbox(label="Summarized Text"),
    title="Free AI Text Summarizer",
    description="Uses Hugging Face's T5 model to generate text summaries.",
)

# Launch the Gradio app
interface.launch(share=True)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d64bb3a62a42a00c3c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


CHATBOT MODEL


In [ ]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
import gradio as gr

# Load pre-trained chatbot model
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# Function for chatbot response
def chatbot_response(user_input):
    inputs = tokenizer(user_input, return_tensors="pt")
    reply_ids = model.generate(**inputs)
    response = tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    return response

# Gradio interface
interface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Type your message..."),
    outputs="text",
    title="AI Chatbot",
    description="A simple AI-powered chatbot using a pre-trained BlenderBot model."
)

# Launch the chatbot
interface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cc5053dae5cfbb46a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
